In [ ]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Hacer clic en la primera caja de inmueble
try:
    primera_caja_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".ad-preview"))
    )
    primera_caja_inmueble.click()
    time.sleep(random.uniform(10, 12))  # Espera con tiempo aleatorio para evitar detección
except Exception as e:
    print("Error al hacer clic en la primera caja de inmueble:", e)
    browser.quit()

# Lista para almacenar los datos
all_data = []

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while True:
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:


            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Localización': localizacion,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Compra'  # Se añade la operación como compra
            }

            all_data.append(data)

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {len(all_data)}: {data}")

            # Pausa de 60 segundos cada 500 inmuebles
            if len(all_data) % 500 == 0:
                print("Pausa de 60 segundos...")
                time.sleep(60)

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation-arrow--next"))
            )
            siguiente_boton.click()

            # Pausa aleatoria para simular el comportamiento humano y evitar bloqueo
            time.sleep(random.uniform(10, 13))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", imprime el error y termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        # Captura errores al cargar la página o al extraer datos y continúa
        print(f"Error al cargar la página del inmueble: {e}")
        continue

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('data_todos_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()

In [1]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Hacer clic en la primera caja de inmueble
try:
    primera_caja_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".ad-preview"))
    )
    primera_caja_inmueble.click()
    time.sleep(random.uniform(10, 12))  # Espera con tiempo aleatorio para evitar detección
except Exception as e:
    print("Error al hacer clic en la primera caja de inmueble:", e)
    browser.quit()

# Lista para almacenar los datos
all_data = []

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while len(all_data) < 50:  # Limitar a 50 inmuebles
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Localización': location,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Compra'  # Se añade la operación como compra
            }

            all_data.append(data)

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {len(all_data)}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation-arrow--next"))
            )
            siguiente_boton.click()

            # Pausa aleatoria para simular el comportamiento humano y evitar bloqueo
            time.sleep(random.uniform(10, 13))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", imprime el error y termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        # Captura errores al cargar la página o al extraer datos y continúa
        print(f"Error al cargar la página del inmueble: {e}")
        continue

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('data_50_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Error al extraer datos de https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/: name 'location' is not defined


KeyboardInterrupt: 

In [3]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Hacer clic en la primera caja de inmueble
try:
    primera_caja_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".ad-preview"))
    )
    primera_caja_inmueble.click()
    time.sleep(random.uniform(10, 12))  # Espera con tiempo aleatorio para evitar detección
except Exception as e:
    print("Error al hacer clic en la primera caja de inmueble:", e)
    browser.quit()

# Lista para almacenar los datos
all_data = []

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while len(all_data) < 50:  # Limitar a 50 inmuebles
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Dirección
            direccion = soup_inmueble.find('h1').text if soup_inmueble.find('h1') else "Dirección no disponible"

            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Extraer latitud y longitud del atributo data-url
            location_div = soup_inmueble.find('div', {'class': 'location'})
            if location_div and 'data-url' in location_div.attrs:
                location_url = location_div['data-url']
                lat, lng = None, None
                # Buscar latitud y longitud en la URL
                for param in location_url.split('&'):
                    if 'latitude' in param:
                        lat = param.split('=')[1]
                    elif 'longitude' in param:
                        lng = param.split('=')[1]
                location = f"Latitud: {lat}, Longitud: {lng}" if lat and lng else "No disponible"
            else:
                location = "No disponible"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Dirección': direccion,
                'Localización': location,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Compra'  # Se añade la operación como compra
            }

            all_data.append(data)

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {len(all_data)}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation-arrow--next"))
            )
            siguiente_boton.click()

            # Pausa aleatoria para simular el comportamiento humano y evitar bloqueo
            time.sleep(random.uniform(10, 13))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", imprime el error y termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        # Captura errores al cargar la página o al extraer datos y continúa
        print(f"Error al cargar la página del inmueble: {e}")
        continue

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('data_50_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/', 'Dirección': 'Casa unifamiliar en alquiler en Calle XIX', 'Localización': 'No disponible', 'Precio': '4.000', 'Superficie Construida': '600', 'Última Actualización': 'Anuncio actualizado el 28/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '600 m²'), ('Superficie útil:', '600 m²'), ('Habitaciones:', '6'), ('Baños:', '7'), ('Tipo de casa:', 'Unifamiliar'), ('Antigüedad:', 'Entre 5 y 10 años'), ('Conservación:', 'En buen estado'), ('Referencia:', '2805-002317')], 'Tipo de operación': 'Compra'}
Error al extraer datos de https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/: 'NoneType' object has no attribute 'get_text'
Inmueble 2: {'Enlace': 'https://www.pisos.com/alquilar/piso-zona_avenida_de_europa28224-45068829832_101800/', 'Dirección': 'Piso en alquiler en Zona Avenida de Europa', 'Localización'

KeyboardInterrupt: 

In [4]:
import random
import time
import pandas as pd
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import undetected_chromedriver as uc

# Configura el navegador con undetected_chromedriver y un User-Agent aleatorio
options = uc.ChromeOptions()
user_agents = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36",
]
options.add_argument(f"user-agent={random.choice(user_agents)}")
browser = uc.Chrome(options=options)

# URL inicial: Página de listados de Madrid
url = "https://www.pisos.com/alquiler/pisos-madrid/"
browser.get(url)

# Manejar las cookies
try:
    WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="didomi-notice-agree-button"]'))
    ).click()
except Exception as e:
    print("No se encontró el botón de cookies o ya fue aceptado.")
time.sleep(5)

# Espera a que los inmuebles se carguen en la página principal
try:
    WebDriverWait(browser, 20).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.ad-preview'))
    )
except Exception as e:
    print("TimeoutException: No se encontraron los inmuebles en el tiempo esperado.")
    browser.quit()

# Hacer clic en la primera caja de inmueble
try:
    primera_caja_inmueble = WebDriverWait(browser, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, ".ad-preview"))
    )
    primera_caja_inmueble.click()
    time.sleep(random.uniform(10, 12))  # Espera con tiempo aleatorio para evitar detección
except Exception as e:
    print("Error al hacer clic en la primera caja de inmueble:", e)
    browser.quit()

# Lista para almacenar los datos
all_data = []

# Bucle para iterar desde el primer inmueble a los siguientes inmuebles desde su página
while len(all_data) < 50:  # Limitar a 50 inmuebles
    try:
        # Extraer la información del inmueble actual
        html_inmueble = browser.page_source
        soup_inmueble = bs(html_inmueble, 'lxml')

        # Extraer datos del inmueble
        try:
            # Dirección
            direccion = soup_inmueble.find('h1').text if soup_inmueble.find('h1') else "Dirección no disponible"

            # Zona
            zona = soup_inmueble.find('p').text if soup_inmueble.find('p') else "Zona no disponible"

            # Precio
            precio = soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}).text.split(' ')[0] if soup_inmueble.find('div', {'class': 'price__value jsPriceValue'}) else "N/A"

            # Superficie construida
            superficie_construida = soup_inmueble.find('span', {'class': 'features__value'}).text.split(' ')[0] if soup_inmueble.find('span', {'class': 'features__value'}) else "N/A"

            # Última actualización
            ultima_actualizacion = soup_inmueble.find('p', {'class': 'last-update__date'}).text if soup_inmueble.find('p', {'class': 'last-update__date'}) else "N/A"

            # Características
            c1 = soup_inmueble.find('div', {'class': 'features__content'})
            features_list = []
            if c1:
                features = c1.find_all('div', {'class': 'features__feature'})
                for feature in features:
                    label = feature.find('span', {'class': 'features__label'}).get_text(strip=True)
                    value = feature.find('span', {'class': 'features__value'}).get_text(strip=True)
                    features_list.append((label, value))

            # Certificado energético
            try:
                energy_certificate = soup_inmueble.find('div', {'class': 'details__block energy-certificate'})
                consumo = energy_certificate.find('div', {'class': 'energy-certificate__data'}).find_all('span')[1].get_text(strip=True)
                emisiones = energy_certificate.find_all('div', {'class': 'energy-certificate__data'})[1].find_all('span')[1].get_text(strip=True)
            except AttributeError:
                consumo = "N/A"
                emisiones = "N/A"

            # Guarda los datos en una lista
            data = {
                'Enlace': browser.current_url,
                'Dirección': direccion,
                'Zona': zona,
                'Precio': precio,
                'Superficie Construida': superficie_construida,
                'Última Actualización': ultima_actualizacion,
                'Consumo Energético': consumo,
                'Emisiones CO2': emisiones,
                'Características': features_list,
                'Tipo de operación': 'Compra'  # Se añade la operación como compra
            }

            all_data.append(data)

            # Imprime los datos de la página actual (opcional)
            print(f"Inmueble {len(all_data)}: {data}")

        except Exception as e:
            print(f"Error al extraer datos de {browser.current_url}: {e}")

        # Intenta hacer clic en el botón de "Siguiente" para ir al siguiente inmueble
        try:
            siguiente_boton = WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.CSS_SELECTOR, ".navigation-arrow--next"))
            )
            siguiente_boton.click()

            # Pausa aleatoria para simular el comportamiento humano y evitar bloqueo
            time.sleep(random.uniform(10, 13))

        except Exception as e:
            # Si no se encuentra el botón "Siguiente", imprime el error y termina el bucle
            print("No se encontró el botón 'Siguiente' o ya no hay más inmuebles.")
            break

    except Exception as e:
        # Captura errores al cargar la página o al extraer datos y continúa
        print(f"Error al cargar la página del inmueble: {e}")
        continue

# Una vez terminado el bucle, puedes convertir los datos en un DataFrame de pandas y guardarlos
df = pd.DataFrame(all_data)
print(df)

# Opcional: guarda el DataFrame en un archivo CSV
df.to_csv('data_50_inmuebles.csv', index=False)

# Cierra el navegador
browser.quit()


Inmueble 1: {'Enlace': 'https://www.pisos.com/alquilar/casa_unifamiliar-las_matas_los_penascales-29189878080_280500/', 'Dirección': 'Casa unifamiliar en alquiler en Calle XIX', 'Zona': 'Las Matas-Los Peñascales (Las Rozas de Madrid)', 'Precio': '4.000', 'Superficie Construida': '600', 'Última Actualización': 'Anuncio actualizado el 28/10/2024', 'Consumo Energético': 'N/A', 'Emisiones CO2': 'N/A', 'Características': [('Superficie construida:', '600 m²'), ('Superficie útil:', '600 m²'), ('Habitaciones:', '6'), ('Baños:', '7'), ('Tipo de casa:', 'Unifamiliar'), ('Antigüedad:', 'Entre 5 y 10 años'), ('Conservación:', 'En buen estado'), ('Referencia:', '2805-002317')], 'Tipo de operación': 'Compra'}
Error al extraer datos de https://www.pisos.com/alquilar/piso-la_finca-47557750571_280500/: 'NoneType' object has no attribute 'get_text'
Inmueble 2: {'Enlace': 'https://www.pisos.com/alquilar/piso-zona_avenida_de_europa28224-45068829832_101800/', 'Dirección': 'Piso en alquiler en Zona Avenida d

KeyboardInterrupt: 